In [3]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

%load_ext autoreload
%autoreload 2

In [4]:
df_temporal = pd.read_csv('../data/interim/telecom_market_temporal_patterns.csv')
df_temporal.head(10)

,date,circle,type_of_connection,service_provider,value,total_circle_subscriber,market_share,num_competitors,subscriber_change,month_num,quarter,subscribers_ma_3,subscribers_ma_6,subscribers_ma_12
0,2009-01-01,All India,wireless,AIRCEL,16761397.0,400045117.0,0.041899,15,NaN,1,1,1.676140e+07,1.676140e+07,1.676140e+07
1,2009-02-01,All India,wireless,AIRCEL,17099390.0,413466301.0,0.041356,15,0.020165,2,1,1.693039e+07,1.693039e+07,1.693039e+07
2,2009-03-01,All India,wireless,AIRCEL,18478325.0,429725181.0,0.043000,15,0.080642,3,1,1.744637e+07,1.744637e+07,1.744637e+07
3,2009-04-01,All India,wireless,AIRCEL,19585299.0,441475184.0,0.044363,16,0.059907,4,2,1.838767e+07,1.798110e+07,1.798110e+07
4,2009-05-01,All India,wireless,AIRCEL,20685711.0,452911165.0,0.045673,16,0.056186,5,2,1.958311e+07,1.852202e+07,1.852202e+07
5,2009-06-01,All India,wireless,AIRCEL,21798731.0,464825338.0,0.046897,16,0.053806,6,2,2.068991e+07,1.906814e+07,1.906814e+07
6,2009-07-01,All India,wireless,AIRCEL,23101900.0,479070905.0,0.048222,16,0.059782,7,3,2.186211e+07,2.012489e+07,1.964439e+07
7,2009-08-01,All India,wireless,AIRCEL,24415514.0,494069898.0,0.049417,15,0.056862,8,3,2.310538e+07,2.134425e+07,2.024078e+07
8,2009-09-01,All India,wireless,AIRCEL,25728633.0,509032539.0,0.050544,15,0.053782,9,3,2.441535e+07,2.255263e+07,2.085054e+07
9,2009-10-01,All India,wireless,AIRCEL,27746797.0,525650418.0,0.052786,15,0.078440,10,4,2.596365e+07,2.391288e+07,2.154017e+07


In [5]:
df_temporal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64900 entries, 0 to 64899
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   date                     64900 non-null  object 
 1   circle                   64900 non-null  object 
 2   type_of_connection       64900 non-null  object 
 3   service_provider         64900 non-null  object 
 4   value                    64900 non-null  float64
 5   total_circle_subscriber  64900 non-null  float64
 6   market_share             64900 non-null  float64
 7   num_competitors          64900 non-null  int64  
 8   subscriber_change        63999 non-null  float64
 9   month_num                64900 non-null  int64  
 10  quarter                  64900 non-null  int64  
 11  subscribers_ma_3         64900 non-null  float64
 12  subscribers_ma_6         64900 non-null  float64
 13  subscribers_ma_12        64900 non-null  float64
dtypes: float64(7), int64(3

In [6]:
from src.feature_engineering import create_churn_targets, validate_churn_patterns
df_with_targets = create_churn_targets(df_temporal)
print(f"Churn distribution:\n{df_with_targets['churn_severity'].value_counts()}")

Churn distribution:
churn_severity
None      49303
High      13538
Low        1511
Medium      548
Name: count, dtype: int64


In [7]:
df_validated = validate_churn_patterns(df_with_targets)

Churn rates by operator category:
operator_category
Major       0.316708
Regional    0.148934
Name: churn_binary, dtype: float64
Top 10 circles by churn rate:
circle
Tamil Nadu              0.266385
All India               0.251294
Rajasthan               0.251004
Punjab                  0.240416
Delhi                   0.236003
Gujarat                 0.229102
Mumbai                  0.228974
Andhra Pradesh          0.226888
Karnataka               0.226865
Uttar Pradesh (West)    0.226120
Name: churn_binary, dtype: float64


In [8]:
from src.feature_engineering import create_temporal_features, create_advanced_features, create_business_features, create_competitive_features
df_temporal_features = create_temporal_features(df_validated)
print(f"Temporal features created. New shape: {df_temporal_features.shape}")

Temporal features created. New shape: (64900, 27)


In [9]:
df_final_features = create_advanced_features(df_temporal_features)
print(f"Advanced features created. New shape: {df_final_features.shape}")

Advanced features created. New shape: (64900, 31)


In [10]:
df_competitive_features = create_competitive_features(df_final_features)
print(f"Competitive features created. New shape: {df_competitive_features.shape}")

Competitive features created. New shape: (64900, 34)


In [11]:
df_all_features = create_business_features(df_competitive_features)
print(f"All features created. Final shape: {df_all_features.shape}")

All features created. Final shape: (64900, 38)


In [12]:
df_all_features.to_csv('../data/final/telecom_churn_features.csv', index=False)

In [13]:
df_all_features.columns.tolist()

['date',
 'circle',
 'type_of_connection',
 'service_provider',
 'value',
 'total_circle_subscriber',
 'market_share',
 'num_competitors',
 'subscriber_change',
 'month_num',
 'quarter',
 'subscribers_ma_3',
 'subscribers_ma_6',
 'subscribers_ma_12',
 'churn_binary',
 'churn_severity',
 'operator_category',
 'subscribers_lag_1',
 'subscribers_lag_3',
 'subscribers_lag_6',
 'subscribers_lag_12',
 'mom_growth',
 'yoy_growth',
 'growth_volatility_3',
 'growth_volatility_6',
 'growth_volatility_12',
 'trend_12m',
 'growth_momentum',
 'growth_acceleration',
 'seasonal_factor',
 'seasonally_adjusted_value',
 'market_rank',
 'share_gap_leader',
 'relative_performance',
 'circle_type',
 'is_wireless',
 'market_size_category',
 'operator_geographic_diversity']

In [14]:
df_all_features['operator_category'].value_counts()

operator_category
Regional    38554
Major       26346
Name: count, dtype: int64

In [16]:
df_all_features['date'].head()

0    2009-01-01
1    2009-02-01
2    2009-03-01
3    2009-04-01
4    2009-05-01
Name: date, dtype: object